# import packages and define stimulus protocols

In [ ]:
# using MAT

using MultivariateStats
using DifferentialEquations
using Plots
gr(fmt="png", size=(1600, 600))

include("../src/HHModel.jl")

# stimulus protocol
current_step = (t, param) -> begin
    (param.start < t)&(t < param.dur + param.start) ? param.step + param.holding + param.noise * randn() : param.holding + param.holding + param.noise * randn()
end

## setting up biophysical parameters

In [ ]:
# type one in matlab - sustained

# biophysical model setup ==> usu. for maximum conductance
htk = HHModel.high_voltage_gated_potassium(2.8, phi=0.85)
# ltk = HHModel.low_voltage_gated_potassium(0.65, subtype=:kv1)
# ik = HHModel.hh_potassium(10.0)
ina = HHModel.hh_sodium(20.0)
ih = HHModel.ihcurrent(0.91)
il = HHModel.leakage(0.03)

_model = [htk, ina, ih, il]
_model_sim_cc = HHModel.simpleConductanceModel(_model, current_step, C=0.9); # current clamp

In [ ]:
# type two in matlab -- transient

# biophysical model setup ==> usu. for maximum conductance
htk = HHModel.high_voltage_gated_potassium(2.8, phi=0.85)
ltk = HHModel.low_voltage_gated_potassium(1.1, subtype=:kv1)
# ik = HHModel.hh_potassium(10.0)
ina = HHModel.hh_sodium(13.0)
ih = HHModel.ihcurrent(0.43)
il = HHModel.leakage(0.03)

_model = [htk, ltk, ina, ih, il]
_model_sim_cc = HHModel.simpleConductanceModel(_model, current_step, C=0.9); # current clamp

In [ ]:
# custom parameters
# biophysical model setup ==> usu. for maximum conductance
htk = HHModel.high_voltage_gated_potassium(1.8)
ltk = HHModel.low_voltage_gated_potassium(0.1, subtype=:kv1)
# ik = HHModel.hh_potassium(10.0)
ina = HHModel.hh_sodium(13.0)
ih = HHModel.ihcurrent(0.3)
il = HHModel.leakage(0.2)

_model = [htk, ltk, ina, ih, il]
# _model = [htk, ina, ih, il]
_model_sim_cc = HHModel.simpleConductanceModel(_model, current_step, C=0.9); # current clamp

---
## running the simulation

In [ ]:
# running the simulation
_p = (E=(sodium=82.0, potassium=-81.0, ih=-46.0, leak=-65.0, syn=3.0), 
      stim=(start=500, step=5.0, holding=0, dur=1500, noise=0),)

# ih.h.Vhalf = -60.0
# HHModel.update!(ih)

tspan = (0.0, 2500.0)
v0 = -60.9766
v0 = -72.7030;
u0 = HHModel.setup_init(_model, v0)

prob = ODEProblem(_model_sim_cc, u0, tspan, _p)
@time sol = solve(prob, Tsit5(), reltol=1e-8, abstol=1e-8)

# preview voltage trace
plot(sol, vars=(1), legend=nothing)
# savefig("demo2.svg")

In [ ]:
# preview current
_trange = 400:0.1:1000
_trace = HHModel.current_decompose(sol, _model, _trange, _p)
plot(xlim=(_trange[1], _trange[end]), legend=:right)
for (key, val) in _trace
    if key == "voltage"
        continue
    end
    plot!(_trange, val, label=key, linewidth=3)
end
plot!()
# savefig("demo2_current.svg")

In [ ]:
# export
_export = HHModel.current_decompose(sol, _model, 0.01:0.01:2500, _p)

# _export = Dict("param" => hcat(sol(0.01:0.01:2500).u...)[1, :])
MAT.matwrite("julia_sim_transient_currents.mat", _export)

---

In [ ]:
# preview dynamics

# dynamics between membrane potential and a particular kinetic variable
plot(sol, vars=(1,2), size=(500, 500), legend=nothing)

In [ ]:
_down_array

In [ ]:
# PCA projection
_var_array = hcat(sol(500:0.1:2000).u...)[2:end-1, :]

_down = fit(PCA, _var_array, );

_down_array = transform(_down, _var_array);

plot(_down_array[1,:], _down_array[2, :], size=(500, 500), legend=nothing)

---

In [ ]:
# sweep protocol

_current_steps = -5:0.5:0

plot(legend=nothing, ylim=(-150, 80), xlim=(190, 250))

@time for _i_step in _current_steps
    _p = (E=(sodium=81.27, potassium=-80.78, ih=-26, leak=-65, syn=3), 
      stim=(start=50, step=_i_step, dur=500, holding=0, noise=0),)
    prob = ODEProblem(_model_sim_cc, u0, tspan, _p)
    sol = solve(prob, Tsit5(), dt=0.1)

    plot!(sol, vars=(1), color=:black, alpha=0.3)
end

plot!()

In [ ]:
# voltage clamp
voltage_step = (t, param) -> begin
    (param.start < t)&(t < param.dur + param.start) ? param.step + param.noise * (rand()-0.5) : -60 + param.noise * (rand()-0.5)
end

_model_sim = HHModel.simpleVoltageClamp(_model, voltage_step, C=0.9)

# running the simulation
_p = (E=(sodium=81.27, potassium=-80.78, ih=-26.0, leak=-65.0, syn=3.0), 
      stim=(start=200, step=30, dur=1500, noise=0),)

tspan = (0.0, 2000.0)
v0 = -60.0
u0 = HHModel.setup_init(_model, v0)

prob = ODEProblem(_model_sim, u0, tspan, _p)
@time sol = solve(prob, Tsit5(), dt=0.01)

# preview current
plot(sol, vars=(length(u0)), legend=nothing, xlim=(180,250))
# savefig("demo2.svg")


In [ ]:
# preview current
_trange = 0:0.1:2000
_trace = HHModel.current_decompose(sol, _model, _trange, _p)
plot(xlim=(_trange[1], _trange[end]))
for (key, val) in _trace
    if key == "voltage"
        continue
    end
    plot!(_trange, val, label=key)
end
plot!(xlim=(180, 1000))
# savefig("demo2_current.svg")